### Dependents
***

In [59]:
from matplotlib import dates
from datetime import datetime
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

### Ensure data loads
***

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite", echo=False)

In [3]:
inspector = inspect(engine)
inspector.get_table_names()

['measure', 'stations']

In [4]:
columns = inspector.get_columns('measure')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station VARCHAR
precip FLOAT
tobs INTEGER
date DATETIME


In [5]:
columns = inspector.get_columns('stations')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station VARCHAR
latitude FLOAT
longitude FLOAT
elevation FLOAT
name VARCHAR
country VARCHAR
state VARCHAR


In [6]:
engine.execute('SELECT * FROM measure LIMIT 5').fetchall()

[(1, 'USC00519397', 0.08, 65, '2010-01-01 00:00:00.000000'),
 (2, 'USC00519397', 0.0, 63, '2010-01-02 00:00:00.000000'),
 (3, 'USC00519397', 0.0, 74, '2010-01-03 00:00:00.000000'),
 (4, 'USC00519397', 0.0, 76, '2010-01-04 00:00:00.000000'),
 (5, 'USC00519397', 0.0, 73, '2010-01-06 00:00:00.000000')]

In [7]:
engine.execute('SELECT * FROM stations LIMIT 5').fetchall()

[(1, 'USC00519397', 21.2716, -157.8168, 3.0, 'WAIKIKI 717.2', 'US', 'HI'),
 (2, 'USC00513117', 21.4234, -157.8015, 14.6, 'KANEOHE 838.1', 'US', 'HI'),
 (3, 'USC00514830', 21.5213, -157.8374, 7.0, 'KUALOA RANCH HEADQUARTERS 886.9', 'US', 'HI'),
 (4, 'USC00517948', 21.3934, -157.9751, 11.9, 'PEARL CITY', 'US', 'HI'),
 (5, 'USC00518838', 21.4992, -158.0111, 306.6, 'UPPER WAHIAWA 874.3', 'US', 'HI')]

### Database Creation
***

In [8]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Measure = Base.classes.measure

In [9]:
session = Session(engine)

In [10]:
session.query(func.sum(Measure.precip)).all()

[(2908.1299999999706)]

In [11]:
date_precip = session.query(Measure.date, Measure.precip).\
    filter(Measure.date > '2017-01-01').\
    order_by(Measure.date).all()

In [12]:
date_precip_df = pd.DataFrame(date_precip)

In [13]:
date_precip_df = date_precip_df.set_index('date')

In [18]:
date_precip_df.index

DatetimeIndex(['2017-01-01', '2017-01-01', '2017-01-01', '2017-01-01',
               '2017-01-01', '2017-01-01', '2017-01-02', '2017-01-02',
               '2017-01-02', '2017-01-02',
               ...
               '2017-08-21', '2017-08-21', '2017-08-21', '2017-08-22',
               '2017-08-22', '2017-08-22', '2017-08-23', '2017-08-23',
               '2017-08-23', '2017-08-23'],
              dtype='datetime64[ns]', name='date', length=1397, freq=None)

In [21]:
date_precip_index = date_precip_df.index.strftime('%m/%d/%Y')

In [23]:
date_precip_index

array(['01/01/2017', '01/01/2017', '01/01/2017', ..., '08/23/2017',
       '08/23/2017', '08/23/2017'],
      dtype='<U10')

In [42]:
len(date_precip_index)

1397

In [47]:
xmin = date_precip_index[0]

In [48]:
xmax = date_precip_index[1396]

In [60]:
years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
yearsFmt = mdates.DateFormatter('%Y')

In [58]:
fig, ax = plt.subplots(figsize=(20,7))
sns.barplot(x=date_precip_index, y='precip', data=date_precip_df, ax=ax, hue=None, saturation=.3)
plt.title('Precip by Date', size=20)
plt.xlabel('Date', size=15)
plt.ylabel('Precip', size=15)
sns.set_style("darkgrid")
#ax.xaxis.set_major_locator(days)
ax.xaxis.date()
ax.xaxis.set_major_formatter(dfmt)
#ax.xaxis.set_minor_locator(days)
plt.xticks(rotation=70)
plt.show()

AttributeError: 'XAxis' object has no attribute 'date'

In [ ]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Stations = Base.classes.stations

In [ ]:
# Importing all dependencies
import sqlalchemy
from sqlalchemy import create_engine, MetaData, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, DateTime
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

In [ ]:
# Create an engine to SQLite database call 'hawaii.sqlite'.
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# Create a connection to the engine
conn = engine.connect()

In [ ]:
# Establishing the class/tables including column name, type, length
Base = declarative_base()

class Stations(Base):
    __tablename__ = 'stations'
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    name = Column(String)
    country = Column(String)
    state = Column(String)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

class Measurements(Base):
    __tablename__ = 'measure'
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    precip = Column(Float)
    tobs = Column(Integer)
    date = Column(DateTime(30))
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [ ]:
# Building the classes in the SQLite db
Base.metadata.create_all(engine)

In [ ]:
# Inpecting tables to ensure the create_all command worked
inspector = inspect(engine)

In [ ]:
inspector.get_table_names()

In [ ]:
# Converting the DataFrames into dictionaries
data_stations = stations.to_dict(orient='records')

In [ ]:
data_measures = measures.to_dict(orient='records')

In [ ]:
# Printing the dictionaries
print(data_stations[:5])

In [ ]:
print(data_measures[:5])

In [ ]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [ ]:
table = sqlalchemy.Table('stations', metadata, autoload=True)

In [ ]:
conn.execute(table.delete())

In [ ]:
# Inserting all of the records for station
conn.execute(table.insert(), data_stations)

In [ ]:
# Executing a select statement on the Stations tables
conn.execute("SELECT * FROM stations LIMIT 5").fetchall()

In [ ]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [ ]:
# Building the tables in the SQLite db
table = sqlalchemy.Table('measure', metadata, autoload=True)

In [ ]:
# Clears the tables before inserting records
conn.execute(table.delete())

In [ ]:
# Inserting all of the records for measures
conn.execute(table.insert(), data_measures)

In [ ]:
# Executing a select statement on the Measurement table
conn.execute("SELECT * FROM measure LIMIT 10").fetchall()